In [1]:
from guppylang import guppy
from guppylang.std.builtins import result
from guppylang.std.angles import angle, pi
from guppylang.std.quantum import measure_array, qubit, x, cx, h, v, vdg, rz


@guppy
def main() -> None:
    q = array(qubit() for _ in range(4))
    x(q[0])

    h(q[0])
    v(q[2])
    for i in range(3):
        cx(q[i], q[i+1])
    rz(q[3], -0.224 * pi)
    start = 4
    for i in range(1, 4):
        j = start - i
        i = start - i - 1
        cx(q[i], q[j])
    h(q[0])
    vdg(q[2])

    v(q[0])
    h(q[2])
    for i in range(3):
        cx(q[i], q[i+1])
    rz(q[3], 0.224 * pi)
    start = 4
    for i in range(1, 4):
        j = start - i
        i = start - i - 1
        cx(q[i], q[j])
    vdg(q[0])
    h(q[2])

    cx(q[0], q[1])
    cx(q[2], q[3])

    c = measure_array(q)
    result("c", c)
    i = 0
    for j in range(4):
        if c[j]:
            i += 2**int(j)
    result("USER:INT:symmetry", i)
    if i != 15:
        exit("Symmetry Violated", 42)

In [2]:
hugr = main.compile()

In [3]:
from selene_sim import build, Quest, CircuitExtractor
from selene_sim.result_handling.parse_shot import postprocess_unparsed_stream

runner = build(main.compile(), "panic")
circuit_extractor = CircuitExtractor()
shots, error = postprocess_unparsed_stream(
    runner.run_shots(
        Quest(),
        n_qubits=4,
        n_shots=10,
        parse_results=True,
        event_hook=circuit_extractor
    )
)

In [4]:
shot_number = 0
circuit = circuit_extractor.shots[shot_number].get_user_circuit()

In [5]:
circuit

[Reset q[0]; Reset q[1]; Reset q[2]; Reset q[3]; PhasedX(1, 0) q[0]; PhasedX(3.5, 0.5) q[1]; PhasedX(0.5, 0) q[2]; PhasedX(3.5, 0.5) q[3]; PhasedX(0.5, 1.5) q[0]; PhasedX(3.5, 0.5) q[2]; Rz(1) q[0]; ZZPhase(0.5) q[0], q[1]; Rz(3.5) q[0]; PhasedX(0.5, 1) q[1]; Rz(3.5) q[1]; ZZPhase(0.5) q[1], q[2]; Rz(3.5) q[1]; PhasedX(0.5, 1) q[2]; Rz(3.5) q[2]; ZZPhase(0.5) q[2], q[3]; Rz(3.5) q[2]; PhasedX(0.5, 1) q[3]; Rz(3.5) q[3]; Rz(3.776) q[3]; PhasedX(3.5, 0.5) q[3]; ZZPhase(0.5) q[2], q[3]; Rz(3.5) q[2]; PhasedX(0.5, 1) q[3]; PhasedX(3.5, 0.5) q[2]; Rz(3.5) q[3]; ZZPhase(0.5) q[1], q[2]; PhasedX(3.5, 0.5) q[3]; Rz(3.5) q[1]; PhasedX(0.5, 1) q[2]; PhasedX(3.5, 0.5) q[1]; Rz(3.5) q[2]; ZZPhase(0.5) q[0], q[1]; PhasedX(3.5, 0) q[2]; Rz(3.5) q[0]; PhasedX(0.5, 1) q[1]; PhasedX(0.5, 1.5) q[2]; PhasedX(0.5, 1.5) q[0]; Rz(3.5) q[1]; Rz(1) q[2]; Rz(1) q[0]; PhasedX(3.5, 0.5) q[1]; PhasedX(3.5, 0.5) q[2]; PhasedX(0.5, 0) q[0]; ZZPhase(0.5) q[0], q[1]; Rz(3.5) q[0]; PhasedX(0.5, 1) q[1]; Rz(3.5) q[1]; 

In [6]:
type(circuit)

pytket._tket.circuit.Circuit

In [8]:
from pytket.extensions.qiskit import tk_to_qiskit # only for printing
print(tk_to_qiskit(circuit))

             ┌────────┐  ┌─────────────┐┌───────┐          ┌──────────┐»
q_0: ─|0>────┤ R(π,0) ├──┤ R(π/2,3π/2) ├┤ Rz(π) ├─■────────┤ Rz(7π/2) ├»
          ┌──┴────────┴─┐└─────────────┘└───────┘ │ZZ(π/2) ├──────────┤»
q_1: ─|0>─┤ R(7π/2,π/2) ├─────────────────────────■────────┤ R(π/2,π) ├»
          └─┬──────────┬┘┌─────────────┐                   └──────────┘»
q_2: ─|0>───┤ R(π/2,0) ├─┤ R(7π/2,π/2) ├───────────────────────────────»
          ┌─┴──────────┴┐└─────────────┘                               »
q_3: ─|0>─┤ R(7π/2,π/2) ├──────────────────────────────────────────────»
          └─────────────┘                                              »
c: 4/══════════════════════════════════════════════════════════════════»
                                                                       »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«     ┌──────────┐          ┌──────────┐     